In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import os
import cv2
from sklearn.model_selection import train_test_split

def images(path):
    return os.listdir(path)

def extract():
    base_path = r"/kaggle/input/fruit-and-vegetable-disease-healthy-vs-rotten/Fruit And Vegetable Diseases Dataset"
    
    for folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder)
        image_files = images(folder_path)
        
        
        train, test = train_test_split(image_files, test_size=0.2, random_state=42)
        train, valid = train_test_split(train, test_size=0.2, random_state=42)
        
        
        train_dir = os.path.join(r"/kaggle/working/", 'train', folder)
        valid_dir = os.path.join(r"/kaggle/working/", 'valid', folder)
        test_dir = os.path.join(r"/kaggle/working/", 'test', folder)
        
        os.makedirs(train_dir, exist_ok=True)
        os.makedirs(valid_dir, exist_ok=True)
        os.makedirs(test_dir, exist_ok=True)
        
        
        for img_name in train:
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)
            save_path = os.path.join(train_dir, img_name)
            cv2.imwrite(save_path, img)
        
      
        for img_name in valid:
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)
            save_path = os.path.join(valid_dir, img_name)
            cv2.imwrite(save_path, img)
        
       
        for img_name in test:
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)
            save_path = os.path.join(test_dir, img_name)
            cv2.imwrite(save_path, img)

extract()


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


In [4]:
import numpy as np
import pandas as pd
import tensorflow,keras
import cv2
from matplotlib import pyplot as plt
import os

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
train_gen_obj=ImageDataGenerator(rotation_range=45,width_shift_range=0.2,height_shift_range=0.5,brightness_range=(0,0.8),\
                                zoom_range=0.4,rescale=1./255)
test_gen_obj=ImageDataGenerator(rescale=1./255)
valid_gen_obj=ImageDataGenerator(rescale=1./255)

In [7]:
train_gen=train_gen_obj.flow_from_directory(directory=r"/kaggle/working/train",\
                target_size=(500,500),classes=None,
                color_mode='rgb',class_mode="categorical",
                batch_size=32)
#bettr to go with gray scale because all colors are same so prefer edges,one hot encoded
#use categorical or sparse ( one hot encoded or sparse)


test_gen=test_gen_obj.flow_from_directory(directory=r"/kaggle/working/test",\
                target_size=(500,500),classes=None,
                color_mode='rgb',class_mode="categorical",
                batch_size=32)


valid_gen=valid_gen_obj.flow_from_directory(directory=r"/kaggle/working/valid",\
                target_size=(500,500),classes=None,
                color_mode='rgb',class_mode="categorical",
                batch_size=32)

Found 18725 images belonging to 28 classes.
Found 5860 images belonging to 28 classes.
Found 4692 images belonging to 28 classes.


In [8]:
train_gen.classes

array([ 0,  0,  0, ..., 27, 27, 27], dtype=int32)

In [9]:
from keras.models import Sequential
from keras.layers import Dense,InputLayer,Conv2D,MaxPool2D,Flatten

In [10]:
model=Sequential()

model.add(InputLayer(shape=(500,500,3)))

model.add(Conv2D(50,(3,3),strides=(1,1),padding="same",activation="relu",use_bias=True))
model.add(Conv2D(20,(3,3),strides=(1,1),padding="same",activation="relu",use_bias=True))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),padding="valid"))

model.add(Conv2D(50,(3,3),strides=(1,1),padding="valid",activation="relu",use_bias=True))


model.add(Conv2D(30,(3,3),strides=(1,1),padding="valid",activation="relu",use_bias=True))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),padding="valid"))

model.add(Conv2D(50,(3,3),strides=(1,1),padding="valid",activation="relu",use_bias=True))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),padding="valid"))


model.add(Flatten())
model.add(Dense(100,activation="relu"))
model.add(Dense(60,activation="relu"))
model.add(Dense(30,activation="relu"))
model.add(Dense(28,activation="softmax"))

In [11]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [12]:
train_gen.batch_size

32

In [15]:
train_gen.class_indices

{'Apple__Healthy': 0,
 'Apple__Rotten': 1,
 'Banana__Healthy': 2,
 'Banana__Rotten': 3,
 'Bellpepper__Healthy': 4,
 'Bellpepper__Rotten': 5,
 'Carrot__Healthy': 6,
 'Carrot__Rotten': 7,
 'Cucumber__Healthy': 8,
 'Cucumber__Rotten': 9,
 'Grape__Healthy': 10,
 'Grape__Rotten': 11,
 'Guava__Healthy': 12,
 'Guava__Rotten': 13,
 'Jujube__Healthy': 14,
 'Jujube__Rotten': 15,
 'Mango__Healthy': 16,
 'Mango__Rotten': 17,
 'Orange__Healthy': 18,
 'Orange__Rotten': 19,
 'Pomegranate__Healthy': 20,
 'Pomegranate__Rotten': 21,
 'Potato__Healthy': 22,
 'Potato__Rotten': 23,
 'Strawberry__Healthy': 24,
 'Strawberry__Rotten': 25,
 'Tomato__Healthy': 26,
 'Tomato__Rotten': 27}

In [16]:
history = model.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // train_gen.batch_size,
    validation_data=valid_gen,
    validation_steps=valid_gen.samples // valid_gen.batch_size,
    epochs=20)

Epoch 1/20
585/585 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1255 - loss: 2.9181

/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
W0000 00:00:1719203229.257503     135 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


585/585 ━━━━━━━━━━━━━━━━━━━━ 1496s 3s/step - accuracy: 0.1255 - loss: 2.9178 - val_accuracy: 0.1965 - val_loss: 2.5770
Epoch 2/20
  1/585 ━━━━━━━━━━━━━━━━━━━━ 4:14 435ms/step - accuracy: 0.1875 - loss: 2.7062

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-06-24 04:28:15.810245: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[20,20,500,500]{3,2,1,0}, u8[0]{0}) custom-call(f32[20,50,500,500]{3,2,1,0}, f32[20,50,3,3]{3,2,1,0}, f32[20]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2024-06-24 04:28:16.079363: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.269210109s
Trying algorithm eng0{} for conv (f32[20,20,500,500]{3,2,1,0}, u8[0]{0}) c

585/585 ━━━━━━━━━━━━━━━━━━━━ 28s 46ms/step - accuracy: 0.1875 - loss: 2.7062 - val_accuracy: 0.1500 - val_loss: 3.0413
Epoch 3/20
585/585 ━━━━━━━━━━━━━━━━━━━━ 1475s 2s/step - accuracy: 0.1902 - loss: 2.5819 - val_accuracy: 0.2806 - val_loss: 2.2742
Epoch 4/20
585/585 ━━━━━━━━━━━━━━━━━━━━ 1s 145us/step - accuracy: 0.1875 - loss: 2.7685 - val_accuracy: 0.2500 - val_loss: 2.3464
Epoch 5/20
585/585 ━━━━━━━━━━━━━━━━━━━━ 1473s 2s/step - accuracy: 0.2546 - loss: 2.3638 - val_accuracy: 0.3515 - val_loss: 2.1233
Epoch 6/20
585/585 ━━━━━━━━━━━━━━━━━━━━ 1s 143us/step - accuracy: 0.1875 - loss: 2.3388 - val_accuracy: 0.2500 - val_loss: 2.0346
Epoch 7/20
585/585 ━━━━━━━━━━━━━━━━━━━━ 1470s 2s/step - accuracy: 0.3214 - loss: 2.1503 - val_accuracy: 0.4030 - val_loss: 1.9614
Epoch 8/20
585/585 ━━━━━━━━━━━━━━━━━━━━ 1s 145us/step - accuracy: 0.4688 - loss: 1.8701 - val_accuracy: 0.4500 - val_loss: 1.9864
Epoch 9/20
585/585 ━━━━━━━━━━━━━━━━━━━━ 1464s 2s/step - accuracy: 0.3881 - loss: 1.9436 - val_accurac

In [17]:
model.save(r'/kaggle/working/my_model_53.h5')


In [18]:
os.listdir(r'/kaggle/working/')

['my_model_53.h5', 'valid', 'train', '.virtual_documents', 'test']